In [ ]:
from pyspark.sql import SparkSession

from pyspark.sql import Row
from pyspark.sql.types import IntegerType, StringType
from pyspark.sql.functions import lit

In [ ]:
spark = (SparkSession.builder
                  .appName("Spark SQL Query Dataframes")
                  .getOrCreate())

In [ ]:
data_path = './data'
file_path = f'{data_path}/utilization.json'
sdf = (spark.read
        .format("json")
        .load(file_path))


In [ ]:
sdf.show(3)

In [ ]:
sdf.printSchema()

In [ ]:
sdf.count()

In [ ]:
# Create a Spark SQL Table and giving it a name to be used in queries

sdf.createOrReplaceTempView("utilization")

In [ ]:
sdf_sql = spark.sql("SELECT * FROM utilization LIMIT 3")

In [ ]:
sdf_sql.show()

In [ ]:
sdf_sql = spark.sql("SHOW COLUMNS FROM utilization")
sdf_sql.show(3)

In [ ]:
sdf_sql = spark.sql("DESCRIBE utilization")
sdf_sql.show(3)

In [ ]:
sdf_sql.count()

In [ ]:
sdf_sql = spark.sql("SELECT server_id, session_count FROM utilization LIMIT 3")
sdf_sql.show()

In [ ]:
sdf_sql = spark.sql("SELECT server_id as sid, session_count as sc FROM utilization")
sdf_sql.show(3)

In [ ]:
sdf_sql = spark.sql("SELECT * FROM utilization WHERE server_id = 120")
sdf_sql.show(3)

In [ ]:
sdf_sql = spark.sql("SELECT server_id, session_count FROM utilization WHERE session_count > 70")
sdf_sql.show(3)

In [ ]:
sdf_sql = spark.sql("SELECT server_id, session_count FROM utilization WHERE session_count > 70 AND server_id = 120")
sdf_sql.show(3)

In [ ]:
sdf_sql = spark.sql("""
                   SELECT server_id, session_count
                   FROM utilization
                   WHERE session_count > 70 AND server_id = 120
                   ORDER BY session_count DESC
                 """)
sdf_sql.show(3)

In [ ]:
sdf_sql = spark.sql("SELECT count(*) FROM utilization")
sdf_sql.show()

In [ ]:
sdf_sql = spark.sql("SELECT count(*) \
                    FROM utilization \
                    WHERE session_count > 70")
sdf_sql.show()

In [ ]:
sdf_sql = spark.sql("""SELECT server_id, count(*)
                    FROM utilization
                    WHERE session_count > 70
                    GROUP BY server_id""")
sdf_sql.show(3)

In [ ]:
sdf_sql = spark.sql("""
                    SELECT server_id, count(*)
                    FROM utilization
                    WHERE session_count > 70
                    GROUP BY server_id
                    ORDER BY count(*) DESC
                    """)
sdf_sql.show(3)

In [ ]:
sdf_sql = spark.sql("""
                    SELECT server_id, min(session_count), avg(session_count), max(session_count)
                    FROM utilization
                    WHERE session_count > 70
                    GROUP BY server_id
                    ORDER BY count(*) DESC
                """)
sdf_sql.show(3)

In [ ]:
sdf_sql = spark.sql("""
                    SELECT server_id, min(session_count), round(avg(session_count),2), max(session_count)
                    FROM utilization
                    WHERE session_count > 70
                    GROUP BY server_id
                    ORDER BY count(*) DESC
                    """)
sdf_sql.show(3)

In [ ]:
file_path = f'{data_path}/utilization.json'

In [ ]:
df_util = (spark.read
               .format("json")
               .load(file_path))

In [ ]:
df_util.createOrReplaceTempView("utilization")

In [ ]:
df_util.show(3)

In [ ]:
file_path = f'{data_path}/server_name.csv'

In [ ]:
df_server = (spark.read
       .format("csv")
       .option("header", "true")
       .load(file_path))

In [ ]:
df_server.show(3)

In [ ]:
df_server.createOrReplaceTempView("server_name")

In [ ]:
df_count = spark.sql("SELECT DISTINCT server_id FROM utilization ORDER BY server_id")
df_count.show(3)

In [ ]:
spark.sql("SELECT min(server_id), max(server_id) FROM utilization").show()

In [ ]:
spark.sql("SELECT * FROM server_name").show(3)

In [ ]:
sdf_join = spark.sql("""
                         SELECT u.server_id, sn.server_name, u.session_count
                         FROM utilization AS u
                         INNER JOIN server_name AS sn
                         ON sn.server_id = u.server_id
                    """)
sdf_join.show(3)   

In [ ]:
df_dup = spark.sparkContext.parallelize([
                             Row(server_name='101 Server', cpu_utilization=85, session_count=80),
                             Row(server_name='101 Server', cpu_utilization=80, session_count=90),
                             Row(server_name='102 Server', cpu_utilization=85, session_count=80),
                             Row(server_name='102 Server', cpu_utilization=85, session_count=80)
                        ]).toDF()

In [ ]:
df_dup.show()

In [ ]:
df_dup.drop_duplicates().show()

In [ ]:
df_dup.drop_duplicates(['server_name']).show()

In [ ]:
df_util.describe().show()

In [ ]:
df_util.describe('cpu_utilization','free_memory').show()

In [ ]:
df_util.stat.corr('cpu_utilization','free_memory')

In [ ]:
df_util.stat.freqItems(('server_id','session_count')).show()

In [ ]:
spark.sql('SELECT min(cpu_utilization), max(cpu_utilization), stddev(cpu_utilization) FROM utilization').show()

In [ ]:
spark.sql("""
            SELECT server_id, min(cpu_utilization), max(cpu_utilization), stddev(cpu_utilization)
            FROM utilization
            GROUP BY server_id
          """).show(3)

In [ ]:
spark.sql('SELECT server_id, FLOOR(cpu_utilization*100/10) AS bucket FROM utilization').show(3)

In [ ]:
sql_window = """
                SELECT 
                event_datetime, 
                server_id, 
                cpu_utilization,
                avg(cpu_utilization) OVER (PARTITION BY server_id) AS avg_server_util
                FROM utilization
            """

In [ ]:
spark.sql(sql_window).show(3)

In [ ]:
sql_window = """
                SELECT 
                event_datetime, 
                server_id, 
                cpu_utilization,
                avg(cpu_utilization) OVER (PARTITION BY server_id) AS avg_server_util,
                cpu_utilization - avg(cpu_utilization) OVER (PARTITION BY server_id) AS delta_server_util
                FROM utilization
            """
spark.sql(sql_window).show(3)

In [ ]:
sql_window = """
                SELECT 
                event_datetime, 
                server_id, 
                cpu_utilization,
                avg(cpu_utilization) OVER(
                                            PARTITION BY server_id 
                                            ORDER BY event_datetime
                                            ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING
                                        ) AS avg_server_util
                FROM utilization
            """
spark.sql(sql_window).show(3)

In [ ]:
sdf = spark.sparkContext.parallelize(
                            [Row(server_name='101 Server', cpu_utilization=85, session_count=80),
                             Row(server_name='101 Server', cpu_utilization=80, session_count=90),
                             Row(server_name='102 Server', cpu_utilization=85, session_count=40),
                             Row(server_name='103 Server', cpu_utilization=70, session_count=80),
                             Row(server_name='104 Server', cpu_utilization=60, session_count=80)]
                            ).toDF()

In [ ]:
df.show()

In [ ]:
df_na = df.withColumn('na_col', lit(None).cast(StringType()))

In [ ]:
df_na.show()

In [ ]:
df_na.fillna('A').show()

In [ ]:
df2 = df_na.fillna('A').union(df_na)

In [ ]:
df2.show()

In [ ]:
df2.na.drop().show()

In [ ]:
df2.createOrReplaceTempView("na_table")

In [ ]:
spark.sql("SELECT * FROM na_table").show()

In [ ]:
spark.sql("SELECT * FROM na_table WHERE na_col IS NULL").show()

In [ ]:
spark.sql("SELECT * FROM na_table WHERE na_col IS NOT NULL").show()

In [ ]:
# Load trainsched.txt
df = spark.read.csv("./data/trainsched.txt", header=True)

# Create temporary table called schedule
"""
A dataframe can be used to create a temporary table. A temporary table is one that 
will not exist after the session ends. Spark documentation also refers to this type 
of table as a SQL temporary view. In the documentation this is referred to as to 
register the dataframe as a SQL temporary view. This command is called on the dataframe 
itself, and creates a table if it does not already exist, replacing it with the current 
data from the dataframe if it does already exist.
"""

df.createOrReplaceTempView("schedule")

In [ ]:
# Inspect the columns in the table df
spark.sql("DESCRIBE schedule").show()

In [ ]:
# Add col running_total that sums diff_min col in each group
sql_window = """
                SELECT 
                    train_id, 
                    station, 
                    time,
                    LEAD(time,1) OVER (PARTITION BY train_id ORDER BY time ASC) AS time_next
                FROM schedule
            """
# Run the query and display the result
spark.sql(sql_window).show()

In [ ]:
# Add col running_total that sums diff_min col in each group
sql_window = """
                SELECT 
                    train_id, 
                    station, 
                    time,
                    diff_min,
                    SUM(diff_min) OVER (PARTITION BY train_id ORDER BY time ASC) AS running_total
                FROM schedule
            """
# Run the query and display the result
spark.sql(sql_window).show()

In [ ]:
# Add col running_total that sums diff_min col in each group
sql_window = """
                SELECT 
                    ROW_NUMBER() OVER (ORDER BY time ASC) AS row,
                    train_id, 
                    station, 
                    time,
                    LEAD(time,1) OVER (ORDER BY time ASC) AS time_next,
                    diff_min,
                    SUM(diff_min) OVER (PARTITION BY train_id ORDER BY time ASC) AS running_total
                FROM schedule
            """
# Run the query and display the result
spark.sql(sql_window).show()

In [ ]:
# Give the identical result in each command
spark.sql("""
                SELECT 
                    train_id, MIN(time) AS start 
                    FROM schedule 
                    GROUP BY train_id
            """).show()


In [ ]:
(df.groupBy('train_id')
        .agg({'time':'min'})
        .withColumnRenamed('min(time)', 'start')
    ).show()

In [ ]:
# Give the identical result in each command
spark.sql("""
                SELECT 
                    train_id, MIN(time), MAX(time)
                    FROM schedule 
                    GROUP BY train_id
            """).show()


In [ ]:
(df.groupBy('train_id')
        .agg({'time':'min', 'time':'max'})
        .withColumnRenamed('min(time)', 'start')
    ).show()

In [ ]:
# Write a SQL query giving a result identical to dot_df
spark.sql("""
                SELECT 
                    train_id, MIN(time) AS start, MAX(time) AS end 
                FROM schedule 
                GROUP BY train_id
            """)

sql_df = spark.sql(query)

sql_df.show()

In [ ]:
from pyspark.sql.functions import lead
from pyspark.sql import Window

# Obtain the identical result using dot notation 
dot_df = df.withColumn(
                        'time_next', 
                        lead('time', 1).over(
                                                Window.partitionBy('train_id').orderBy('time')
                                            )
                      )


dot_df.show()

In [ ]:
# Convert window function from dot notation to SQL
from pyspark.sql.functions import unix_timestamp


dot_df = df.withColumn(
                        'diff_min', 
                        (
                            unix_timestamp(
                                        lead('time', 1).over(Window.partitionBy('train_id').orderBy('time')),
                                        'H:m') -
                            unix_timestamp('time', 'H:m')
                        )/60
)

dot_df.show()

In [ ]:
# Write a SQL query giving a result identical to dot_df
spark.sql("""
                SELECT 
                    *, 
                    (
                        UNIX_TIMESTAMP(LEAD(time, 1) OVER (PARTITION BY train_id ORDER BY time),'H:m') 
                         - UNIX_TIMESTAMP(time, 'H:m')
                    )/60 AS diff_min 
                FROM schedule 
            """)

sql_df = spark.sql(query)

sql_df.show()

In [ ]:
# Load the dataframe
df = spark.read.load('./data/sherlock_sentences.parquet')

# Filter and show the first 5 rows
df.where('id > 70').show(5, truncate=False)

In [ ]:
from pyspark.sql.functions import split, explode


# Split the clause column into a column called words 
split_df = clauses_df.select(split('clause', ' ').alias('words'))
split_df.show(5, truncate=False)

# Explode the words column into a column called word 
exploded_df = split_df.select(explode('words').alias('word'))
exploded_df.show(10)

# Count the resulting number of rows in exploded_df
print("\nNumber of rows: ", exploded_df.count())

In [ ]:
# Word for each row, previous two and subsequent two words

sql_query = """
                SELECT
                        part,
                        LAG(word, 2) OVER(PARTITION BY part ORDER BY id ASC) AS w1,
                        LAG(word, 1) OVER(PARTITION BY part ORDER BY id ASC) AS w2,
                        word AS w3,
                        LEAD(word, 1) OVER(PARTITION BY part ORDER BY id ASC) AS w4,
                        LEAD(word, 2) OVER(PARTITION BY part ORDER BY id ASC) AS w5
                FROM text
"""

spark.sql(sql_query).where("part = 12").show(10)

In [ ]:
# Determine that there are 12 chapters by the following:

(text_df.select('chapter')
       .distinct()
       .sort('chapter')
       .show(truncate=False))

In [ ]:
# Repartition the text_df into 12 partitions, with each chapter in its own partition.
repart_df = text_df.repartition(12, 'chapter')

# Display the number of partitions in the new dataframe.
repart_df.rdd.getNumPartitions()

In [ ]:
# Find the top 10 sequences of five words

sql_query = """
                SELECT w1, w2, w3, w4, w5, COUNT(*) AS freq FROM (
                   SELECT 
                       word AS w1,
                       LEAD(word,1) OVER(PARTITION BY part ORDER BY id ) AS w2,
                       LEAD(word,2) OVER(PARTITION BY part ORDER BY id ) AS w3,
                       LEAD(word,3) OVER(PARTITION BY part ORDER BY id ) AS w4,
                       LEAD(word,4) OVER(PARTITION BY part ORDER BY id ) AS w5
                   FROM text
                )
                GROUP BY w1, w2, w3, w4, w5
                ORDER BY count DESC
                LIMIT 10
            """ 


df = spark.sql(sql_query)

df.show()

In [ ]:
# Unique 5-tuples sorted alphabetically in descending order

sql_query = """
                SELECT DISTINCT w1, w2, w3, w4, w5 FROM (
                   SELECT 
                       word AS w1,
                       LEAD(word,1) OVER(PARTITION BY part ORDER BY id ) AS w2,
                       LEAD(word,2) OVER(PARTITION BY part ORDER BY id ) AS w3,
                       LEAD(word,3) OVER(PARTITION BY part ORDER BY id ) AS w4,
                       LEAD(word,4) OVER(PARTITION BY part ORDER BY id ) AS w5
                   FROM text
                )
                ORDER BY w1 DESC, w2 DESC, w3 DESC, w4 DESC, w5 DESC 
                LIMIT 10
            """ 


df = spark.sql(sql_query)

df.show()

In [ ]:
#   Most frequent 3-tuple per chapter

sql_query = """
                SELECT chapter, w1, w2, w3, count FROM
                    (
                      SELECT
                      chapter,
                      ROW_NUMBER() OVER (PARTITION BY chapter ORDER BY count DESC) AS row,
                      w1, w2, w3, count
                      FROM ( %s )
                    )
                WHERE row = 1
                ORDER BY chapter ASC
            """ % subquery

spark.sql(sql_query).show()

In [ ]:
"""
A dataframe df1 is loaded from a csv file. Several processing steps are performed on it. As df1 is to 
be used more than once, it is a candidate for caching.

A second dataframe df2 is created by performing additional compute-intensive steps on df1. It is also 
a candidate for caching.

Because df2 depends on df1 the question arises: is it better to cache df1, or to cache df2?
"""

# Unpersists df1 and df2 and initializes a timer
prep(df1, df2) 

# Cache df1
df1.cache()


# Run actions on both dataframes
run(df1, "df1_1st") 
run(df1, "df1_2nd")
run(df2, "df2_1st")
run(df2, "df2_2nd", elapsed=True)


# Prove df1 is cached
print(df1.is_cached)


# Unpersist df1 and df2 and initializes a timer
prep(df1, df2) 

# Persist df2 using memory and disk storage level 
df2.persist(storageLevel=pyspark.StorageLevel.MEMORY_AND_DISK)

# Run actions both dataframes
run(df1, "df1_1st") 
run(df1, "df1_2nd")
run(df2, "df2_1st")
run(df2, "df2_2nd", elapsed=True)


# Cache df1, because it improves the time of the 2nd, 3rd, and 4th action.

In [ ]:
"""
Caching and uncaching tables:

A dataframe is cached using a cache() or persist() operation, a table is cached using a cacheTable() operation.

"""


# List the tables
print("Tables:\n", spark.catalog.listTables())

# Cache table1 and Confirm that it is cached
spark.catalog.cacheTable('table1')
print("table1 is cached: ", spark.catalog.isCached('table1'))

# Uncache table1 and confirm that it is uncached
spark.catalog.uncacheTable('table1')
print("table1 is cached: ", spark.catalog.isCached('table1'))

In [ ]:
# Spark UI is available on http://localhost:4040/jobs/

In [ ]:
spark.catalog.cacheTable('schedule')

In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG, format='%(levelname)s - %(message)s')

In [ ]:
# Log columns of text_df as debug message
logging.debug(f"df columns: {df.columns}")

# Log whether table1 is cached as info message
logging.info(f'schedule is cached: {spark.catalog.isCached(tableName="schedule")}')

# Log first row of text_df as warning message
logging.warning(f'The first row of text_df:\n {df.first()}')

# Log selected columns of text_df as error message
logging.error(f'Selected columns: {df.select("id", "word")}')

In [ ]:
# Uncomment the 5 statements that do NOT trigger df

logging.debug("df columns: %s", df.columns)

logging.info("schedule is cached: %s", spark.catalog.isCached(tableName="schedule"))

# logging.warning("The first row of df: %s", df.first())

logging.error("Selected columns: %s", df.select("id", "word"))

logging.info("Tables: %s", spark.sql("SHOW tables").collect())

logging.debug("First row: %s", spark.sql("SELECT * FROM schedule LIMIT 1"))

# logging.debug("Count: %s", spark.sql("SELECT COUNT(*) AS count FROM schedule").collect())

In [ ]:
logging.basicConfig(stream=sys.stdout, level=logging.ERROR, format='%(levelname)s - %(message)s')

In [ ]:
# Run explain on text_df
df.explain()

In [ ]:
# Run explain on "SELECT COUNT(*) AS count FROM table1" 
spark.sql("SELECT COUNT(*) AS count FROM schedule").explain()

In [ ]:
# Run explain on "SELECT COUNT(DISTINCT word) AS words FROM table1"
spark.sql("SELECT COUNT(DISTINCT train_id) AS trains FROM schedule").explain()

In [ ]:
# Sometimes your data needs a transformation that is not supported by built-in functions. 
# This is where a custom user defined function ("UDF") is suitable.

from pyspark.sql.functions import udf
from pyspark.sql.types import BooleanType, StringType, ArrayType, FloatType,

# Returns true if the value is a nonempty vector
nonempty_udf = udf(lambda x:  
    True if (x and hasattr(x, "toArray") and x.numNonzeros())
    else False, BooleanType())

# Returns first element of the array as string
s_udf = udf(lambda x: str(x[0]) if (x and type(x) is list and len(x) > 0) else '', StringType())

# Returns true if the value is a nonempty vector
short_udf = udf(lambda x: True if not x or len(x) < 10 else False, BooleanType())
df.select(short_udf('textdata').alias("is short")).show(3)

# UDF removes items in TRIVIAL_TOKENS from array
rm_trivial_udf = udf(lambda x:
                     list(set(x) - TRIVIAL_TOKENS) if x
                     else x,
                     ArrayType(StringType()))


# Removes last item in array
in_udf = udf(lambda x: x[0:len(x)-1] if x and len(x) > 1 else [],
                ArrayType(StringType())
                )

# Returns first element of the array as Integer
first_udf = udf(
                lambda x: int(x.indices[0])
                    if (x and hasattr(x, "toArray") and x.numNonzeros())
                    else 0,
                IntegerType())

# Selects the first element of a vector column
first_udf = udf(lambda x:
            float(x.indices[0]) 
            if (x and hasattr(x, "toArray") and x.numNonzeros())
            else 0.0,
            FloatType())